In [2]:
import tweepy
import pandas as pd
import numpy as np
import os
import shutil
import glob
from datetime import datetime
import re
import random

In [3]:
consumer_key = "rs8fimR6iW9hQvLg45utLmdkF" # REPLACE WITH OWN CONSUMER KEY AND SECRET
consumer_secret = "z19dkW6zpmzBICreE6C0XGCBLJlw64k7byRi58vHYLvzUEqrbi"
  
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

# calling the api 
api = tweepy.API(auth, wait_on_rate_limit=True)

API Notes:
- User can be suspended, tweet could be deleted -> tweet ID returns error: Forbidden: 403 Forbidden
63 - User has been suspended. -> need exception handling
- Tweepy's `tweet.user.location` doesn't always return county level
- other potential variables of interest: `description` (text of tweet), `followers_count` (to filter for most influential accounts?), `friends_count` (how many following), `created_at`, `id_str` (unique ID for user, could see who's tweeting about COVID the most)

Data Collection Notes:  

0) Download txtcollector to combine csvs for each day into one for month: https://bluefive.pairsite.com/txtcollector.htm
1) Download csvs for month using DownGit: https://downgit.github.io/#/home 
2) combine using txtcollector
3) Run below code

In [4]:
df = pd.read_csv("tweet_data/combinedapr2020.txt") # <- change to correct file name
total_tweets = len(df.index) # around 10 to 30 million tweets per month, maybe sample... 100,000? -> filter those for US, ~10,000 per month
total_tweets

31011963

In [5]:
# sample from all tweets
random.seed(1234)
sample_size = 100000
rand_tweets_indices = random.sample(range(0,total_tweets),sample_size)
tweet_ids = df.iloc[:,0]
tweet_id_sample = tweet_ids[rand_tweets_indices].values
tweet_id_sample

array([1254211431052034054, 1250172852227244032, 1246437946540032000, ...,
       1247011731035095041, 1245724205309714433, 1255383327495905281],
      dtype=int64)

In [6]:
# filter function for US states
state_abbrs = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY','USA','US']
states = ["Alaska", "Alabama", "Arkansas", "American Samoa", 
            "Arizona", "California", "Colorado", "Connecticut", "District of Columbia",
            "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", 
            "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", 
            "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", 
            "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", 
            "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", 
            "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", 
            "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", 
            "Wyoming", "United States"]
            
def filter_location(location):
    if ',' not in location:
        return False
    in_state_abbr = [re.search('^' + re.escape(state_abbr) + '|' + re.escape(state_abbr) + '$',location) for state_abbr in state_abbrs]
    in_state = [re.search('^' + re.escape(state.lower()) + '|' + re.escape(state.lower()) + '$',location.lower()) for state in states]
    return any(in_state_abbr + in_state)

In [28]:
id_t =  '1487802028394901508'
status = api.get_status(id_t)
user = status.user
print(status.favorite_count)

2655


In [58]:
# testing filter function
print(filter_location('Portland, OR'))
print(filter_location('Somewhere, India'))
print(filter_location('USA'))

True
False
False


In [9]:
# loop through all tweets and filter
all_rows = []
columns = ['tweet_id','location','date','likes','retweets','quotes','replies','text']

for i,id in enumerate(tweet_id_sample[:3]):
    # save to csv every 500 tweets scanned in case of failure
    if i % 500 == 0:
        print("tweet",i, ";",round(i/sample_size * 100,4), "pct done")
        US_tweets = pd.DataFrame(all_rows,columns=columns)
        US_tweets.to_csv('tweet_data/US_covid_tweets.csv',index=False) # change this to reflect year_month
    
    # get tweet from id
    try:
        status = api.get_status(id)
        user = status.user
        full = status.retweeted_status
        # only append if user location in US
        locale = user.location
        if filter_location(locale):
            # if('mask' in status.text.lower()):
            row=[id, user.location, user.created_at.strftime("%Y-%m-%d"), 
                full.favorite_count, full.retweet_count, full.quote_count, full.reply_count, 
                full.extended_tweet.get('full_text')]
            all_rows.append(row)
    except: # usually tweet deleted or user suspended
        continue


tweet 0 ; 0.0 pct done
tweet 500 ; 0.5 pct done


Rate limit reached. Sleeping for: 103


tweet 1000 ; 1.0 pct done
tweet 1500 ; 1.5 pct done


Rate limit reached. Sleeping for: 756


tweet 2000 ; 2.0 pct done
tweet 2500 ; 2.5 pct done


...

Rate limit reached. Sleeping for: 762


tweet 98500 ; 98.5 pct done


Rate limit reached. Sleeping for: 759


tweet 99000 ; 99.0 pct done
tweet 99500 ; 99.5 pct done


Rate limit reached. Sleeping for: 755


In [ ]:
# TO RUN THIS AFTER ALL MONTHS COLLECTED

# load existing csvs
# US_tweets = pd.read_csv('US_covid_tweets.csv')


# concat into one big csv and save

In [49]:
# Don't need to run this; just correcting a mistake
valid_locations = US_tweets['location'].apply(lambda x: filter_location(x))
US_tweets = US_tweets.loc[valid_locations]
print(len(US_tweets.index)) # end up with 5.7k tweets
US_tweets.head()

5773


,location,likes,text,date
0,"Minnesota, USA",287188,RT @B52Malmet: CDC confirms second US case of ...,2020-01-24
4,"Chicago, IL",822,RT @ReutersBiz: Coronavirus outbreak may disru...,2020-01-28
5,"California, USA",21186,#Russia has just closed border due to #Coronav...,2020-01-30
9,"The Swamp, DC",88859,RT @AP: BREAKING: Delta Air Lines and American...,2020-01-31
13,"Rockford, IL",480,China Lockdown Spreads To 33 Million People As...,2020-01-24


In [97]:
# same here; fixing
# fix_df = pd.read_csv('tweet_data/US_tweets_2020_01.csv')
# fix_df.head()
num_tweets = len(fix_df.index)
print(num_tweets)

all_rows = []
fixed_cols = ['id','location','text','favorites','retweets','quotes','replies']
# full archive search setup here: https://developer.twitter.com/en/account/environments # MAX 50 retweets
for i,tweet in fix_df.head(1).iterrows():
    if i % 100 == 0:
        print("tweet",i,";",round(i/num_tweets * 100,4), "pct done")
        
    date_str = datetime.strftime(datetime.strptime(tweet.date,"%m/%d/%Y"),"%Y%m%d")
    tweet_text = tweet.text.replace(':','')[:128]
    # print(tweet.location,tweet.date)
    results = api.search_full_archive(label = 'individualTweetSearch', query=tweet_text, fromDate=date_str+'0000',toDate=date_str+'2359')
    for r in results:
        user = r.user
        full = r.retweeted_status
        if user.location == tweet.location: # found match
            # new row
            row = [r.id,user.location,full.extended_tweet.get('full_text'),
                full.favorite_count,full.retweet_count,full.quote_count,full.reply_count]
            all_rows.append(row)
            # break when found
            break
fixed_df = pd.DataFrame(all_rows,columns=fixed_cols)
fixed_df.to_csv('fixed_df.csv',index=False)

5772
tweet 0 ; 0.0 pct done


TooManyRequests: 429 Too Many Requests
Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.